# 1. Import Library

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer
# from langchain_huggingface import HuggingFaceEmbeddings

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [2]:
pd.set_option('display.max_colwidth', 500)

# 2. Preparing Data

In [42]:
full_df = pd.read_csv('../artikel/all_health_articles.csv')
# df.drop(columns=['description', 'content'], inplace=True)
df = full_df[['article_index', 'content']]
df.head()

,article_index,content
0,1,ginjal organ vital fungsi nyaring darah buang limbah jaga imbang cair tubuh jaga fungsi ginjal turun timbul serius sakit ginjal kembang gejala tahap jaga sehat ginjal 5 rawat ginjal sederhana efektif cegah gejala ginjal masalah 1 minum air putih asupan cair bantu ginjal optimal buang racun sisa metabolisme kurang minum air sebab dehidrasi tingkat risiko batu ginjal biasa minum 8 sampai 10 gelas air putih hari jaga fungsi ginjal sehat 2 batas konsumsi garam asupan garam lebih tingkat tekan da...
1,2,jaga sehat mata mata peran aktivitas seharihari dari ajar interaksi lingkung sehat mata abai muncul dari iritasi ringan infeksi sakit serius risiko sebab buta jaga mata periksa rutin dokter biasa pola hidup sehat dukung fungsi lihat optimal iring tambah usia biasa seharihari kurang sehat fungsi mata turun cegah langkah sederhana jaga asupan nutrisi istirahat tatap layar lindung mata dari papar sinar lebih jaga kualitas lihat tahan lebih tips jaga sehat mata mata salah organ aktif hari lihat ...
2,3,sebab campak campak sakit tular sebab virus campak tampek sebar cepat dari orang orang mekanisme utama tular percik air liur droplet dari mulut hidung derita batuk bersin bicara tular sentuh benda kontaminasi virus sentuh hidung mulut mata puluh faktor sebab pengaruh vaksinasi campak mmr campak serang individu vaksin vaksin mmr measles mumps rubella bukti efektif cegah sakit anakanak terima vaksin lengkap milik risiko lebih infeksi milik kebal tubuh virus campak tampek virus usia bayi muda v...
3,4,campak campak rubella sakit tular sebab virus dari keluarga paramyxoviridae genus morbillivirus sakit serang salur napas timbul gejala sistemik demam batuk pilek sakit tenggorok mata merah ruam kulit campak sakit tular pindah dari orang orang udara percik air liur benda kontaminasi virus gejala campak gejala campak muncul inkubasi 7 sampai 14 hari gejala flu demam lemas batuk kering pilek sakit tenggorok diare muntah mata merah air sensitif cahaya bercak putih bintik koplik muncul mulut hari...
4,5,sehat gigi mulut sehat gigi mulut kondisi gigi gusi lidah rongga mulut jaga bersih fungsi mulut gerbang utama makan minum masuk tubuh organ peran proses cerna makan masuk tinggal mulut bersih sisa kembang bakteri sehat mulut kait indah senyum napas segar tentu sehat tubuh akibat jaga sehat gigi mulut abai bersih mulut gigi picu ganggu sehat muncul bau mulut halitosis sisa makan busuk mulut akibat biasa rokok gigi lubang akibat tumpuk plak kikis email gigi radang gusi gingivitis atas kembang ...


# 3. Dense Encoding

In [4]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(df['content'])

d:\Hafiz\Kuliah\Semester 5\Information Retrieval\Projek\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [5]:
print(embeddings)

[[-0.02351626  0.07242429 -0.02842112 ...  0.00293551 -0.07409749
  -0.04062068]
 [ 0.00615982  0.01363388 -0.01907845 ...  0.00788074 -0.07486614
  -0.05919098]
 [-0.01720827  0.01469457 -0.06229497 ... -0.05840543 -0.0243688
   0.02063797]
 ...
 [-0.00958528  0.01784758  0.01170867 ... -0.0413559  -0.04990955
   0.02661984]
 [-0.04443652  0.03816727 -0.0153152  ...  0.00447865  0.02433346
  -0.04069825]
 [ 0.02841109  0.0437499   0.02776248 ... -0.00588526 -0.02467242
  -0.0093273 ]]


In [7]:
embeddings.shape

(420, 384)

# 4. Building a Vector Index

In [18]:
index = faiss.IndexFlatL2(embeddings.shape[1])

In [19]:
index.is_trained

True

In [20]:
index.add(embeddings)

In [21]:
index.ntotal

420

# 5. Query and Retrieval

In [60]:
k = 10
example_query = ["gejala diabetes"]
query = model.encode(example_query)

In [61]:
distances, indices = index.search(query, k)
indices

array([[409, 413,  48,  85,  50, 352, 200,   0, 366,  99]])

In [62]:
print("Top 5 nearest neighbors:", indices)
print("Distances:", distances)

Top 5 nearest neighbors: [[409 413  48  85  50 352 200   0 366  99]]
Distances: [[0.8670476 0.9403507 0.9557108 0.9835525 0.989938  1.0441706 1.044924
  1.0752895 1.0770338 1.084089 ]]


In [63]:
results = full_df[['title', 'content']].iloc[indices[0]]

In [64]:
results = pd.DataFrame(results)
results['similarity score'] = distances[0]

In [65]:
results

,title,content,similarity score
409,gejala diabetes tipe 1 anak minum berat badan turun,diabetes tipe 1 anak lambat nali gejala dari biasa seharihari orangtua lebih peka kondisi lebih cepat deteksi tangan dapat anak makan minum kencing berat badan turun drastis tidakngompolngompollagi anak loyo pikir diabetes prof dr dr aman bhakti pulung spak subsp end faap frcpi hon pakar endokrin anak dari rs cipto mangunkusumo lansir dariantara jumat date,0.867048
413,diabetes tipe 1 anak diagnosis cegah mati,70 persen anak usia 18 indonesia telat diagnosis diabetes tipe 1 diabetes tipe 1 anak perhati serius tanah air 70 persen pasien diagnosis telat ketoasidosis diabetik kad tinggal prof dr dr aman bhakti pulung spa subsp end faap frcpi hon dokter subspesialis endokrinologi anak rs cipto mangunkusumo rscm lansir dariantara kamis date,0.940351
48,5 faktor risiko diabetes tahu,tahu biotizen 91 juta orang indonesia derita diabetes sakit serang kelompok usia 55 74 diabetes alami orang usia muda dari 20an 40an diabetes risiko diabetes kondisi kronis kadar gula darah tubuh akibat ganggu produksi fungsi insulin sayang orang sadar milik diabetes sampai gejala diabetes muncul ganggu aktivitas seharihari ciri ciri diabetes perhati haus lapar buang air malam hari berat badan turun luka sembuh mudah lelah pandang kabur biotizen rasa gejala diabetes konsultasi dokter tangan ...,0.955711
85,gejala diabetes melitus tahu,diabetes mellitus kondisi tanda tinggi kadar gula glukosa darah terusmenerus tubuh produksi insulin insulin efektif insulin hormon ubah glukosa energi tubuh hasil insulin selsel tubuh respons insulin efektif kadar glukosa darah tingkat kenal hiperglikemia salah sehat bahaya temu masyarakat indonesia diabetes tangan din hindar efek lebih buruk hari salah langkah masyarakat tangan din kenal gejala diabetes melitus muncul tubuh jenisjenis diabetes melitus diabetes melitus bagi tipe dasar sebab ...,0.983553
50,bahaya diabetes mellitus faktor risiko tingkat,diabetes mellitus dm salah sehat global serius dampak ancam kualitas hidup derita sakit sebab komplikasi jangka pengaruh sistem tubuh ujung komplikasi akut bahaya nyawa ironis orang sadar bahaya dm komplikasi muncul sebab kurang paham sakit kurang tahu bahaya cegah kelola dm buruk kondisi gilir sebab tingkat dm masyarakat diabetes mellitus kelompok sakit salah sebab utama mati dunia perhati khusus cegah kelola turun angka mati cegah komplikasi timbul komplikasi muncul akibat dm bagi kategori...,0.989938
352,efek samping metformin guna diabetes tipe 2 tips cegah,efek samping metformin alami guna obat diabetes tipe 2 dari gejala ringan kondisi serius tangan paham efek samping metformin kenal lebih muncul keluh metformin obat bantu turun kadar gula darah derita diabetes tipe 2 tingkat respons tubuh insulin efektif pilih utama dokter guna metformin potensi timbul efek samping orang efek samping metformin ringan ganggu cerna sifat serius tangan paham efek samping metformin lebih waspada ambil langkah alami keluh obat efek samping metformin efek samping ...,1.044171
200,hari diabetes dunia diabetasol gelar edukasi sehat,ingat hari diabetes dunia diabetasol gelar rangkai acara talk show 12 daerah online acara 5 negara asia indonesia ingat hari diabetes dunia 14 november 2022 pt sanghiang perkasa kalbe nutritionals brand diabetasol ada live talkshow daring masyarakat acara selenggara 10 sampai 12 november 2022 pause next unmute current time 000 duration 030 loaded fullscreen backward skip 10s play video forward skip 10s rangkai live talkshow diabetasol 12 kota jakarta malang bandung medan yogyakarta palembang...,1.044924
0,5 rawat ginjal sehat cegah sakit ginjal,ginjal organ vital fungsi nyaring darah buang limbah jaga imbang cair tubuh jaga fungsi ginjal turun timbul serius sakit ginjal kembang gejala tahap jaga sehat ginjal 5 rawat ginjal sederhana efektif cegah gejala ginjal masalah 1 minum air putih asupan cair bantu ginjal optimal buang racun sisa metabolisme kurang minum air sebab dehidrasi 